In [5]:
import os

def findAllFiles(path):
    res = []
    path, folders, files = next(os.walk(path))
    if folders:
        for f in folders:
            res.extend(findAllFiles(os.path.join(path, f)))
    res.extend([os.path.join(path, f) for f in files])
    return res

ds_path = "tiny-imagenet-200/"
os.system("mkdir " + ds_path + "dng")


0

In [38]:
import dng
import cfa
import numpy as np
from PIL import Image

suffix         = "train"
save_path      = ds_path + "dng/"
imgs           = findAllFiles(ds_path + suffix)[:10]
cell_x, cell_y = (64,64)
pad_x, pad_y   = ((226 - 64) // 2, (226 - 64) // 2)

for n, img in enumerate(imgs):
    img     = Image.open(img).convert("RGB")
    img_arr = cfa.rgb_to_cfa(np.array(img, dtype=np.uint16) * 2**8, cfa.rgb_kf)
    
    res = np.zeros((pad_x*2 + cell_x, pad_y*2 + cell_y), dtype=np.uint16)
    res[pad_x:pad_x+cell_x, pad_y:pad_y+cell_y] = img_arr
    
    dng.saveFile(res, f"{save_path}{suffix}_{n}")

In [8]:
import os

os.system(f"rawtherapee-cli -Y -b8 -n -p ./tiny -c {save_path}")

81